In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

2024-12-12 06:55:17.476726: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-12 06:55:17.477225: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-12 06:55:17.479120: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-12 06:55:17.484753: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-12 06:55:17.493858: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [ ]:
IMAGE_SIZE = 255
EPOCHS = 10
BATCH_SIZE = 16
CHANNELS=3

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "potato",
    shuffle=True,
    image_size = (IMAGE_SIZE ,IMAGE_SIZE),
    batch_size = BATCH_SIZE 
)


In [ ]:
class_names = dataset.class_names
class_names 

In [ ]:
len(dataset)

In [ ]:
plt.figure(figsize=(20, 20))
for image_batch, label_batch in dataset.take(1):
    for i in range(12):
        ax = plt.subplot(3,4,i+1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[label_batch[i]])
        plt.axis("off")

In [ ]:
train_size = 0.8
len(dataset)

len(dataset)*train_size

train_ds=dataset.take(54)
len(train_ds)

test_ds=dataset.skip(54)
len(test_ds)

val_size=0.1
len(dataset)*val_size

val_ds = test_ds.take(6)
len(val_ds)

test_ds = test_ds.skip(6)
len(test_ds)

In [ ]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000 ):
    ds_size = len(ds)
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)


In [23]:
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.Rescaling(1.0/255)
])


In [25]:
# Define data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
], name="data_augmentation")


In [27]:
model = models.Sequential(name="model")
model.add(layers.Input(shape=(255, 255, 3)))  # Make sure this matches your data input shape
model.add(data_augmentation)
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))  # Adjust the input size if necessary
model.add(layers.Dense(10, activation='softmax'))

# Display the model summary to verify the output shapes
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Model: "model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ data_augmentation (Sequential)  │ (None, 255, 255, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 253, 253, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 126, 126, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 124, 124, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │    51,381,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │        10,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,779,914 (197.52 MB)

 Trainable params: 51,779,914 (197.52 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Fit the model
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    verbose=1,
    validation_data=val_ds
)


Epoch 1/20
285/285 ━━━━━━━━━━━━━━━━━━━━ 211s 729ms/step - accuracy: 0.6098 - loss: 20.3395 - val_accuracy: 0.6339 - val_loss: 0.8604
Epoch 2/20
285/285 ━━━━━━━━━━━━━━━━━━━━ 1653s 6s/step - accuracy: 0.7372 - loss: 0.6551 - val_accuracy: 0.6036 - val_loss: 0.8677
Epoch 3/20
285/285 ━━━━━━━━━━━━━━━━━━━━ 207s 725ms/step - accuracy: 0.7803 - loss: 0.5064 - val_accuracy: 0.8375 - val_loss: 0.3993
Epoch 4/20
285/285 ━━━━━━━━━━━━━━━━━━━━ 206s 724ms/step - accuracy: 0.8470 - loss: 0.3683 - val_accuracy: 0.8500 - val_loss: 0.3348
Epoch 5/20
142/285 ━━━━━━━━━━━━━━━━━━━━ 1:44 731ms/step - accuracy: 0.9034 - loss: 0.2562

In [ ]:
scores = model.evaluate(test_ds)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training Accuracy and Validation Accuracy')

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), acc, label='Training loss')
plt.plot(range(EPOCHS), val_acc, label='Validation loss')
plt.legend(loc='upper right')
plt.title('Training loss and Validation loss')


In [ ]:
import numpy as np
for images_batch, labels_batch in test_ds.take(1):

    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()

    print("first image to predict")
    plt.imshow(first_image)
    print("actual label :", class_names[first_label])

    batch_prediction = model.predict(images_batch)
    print("predicted label :",class_names[np.argmax(batch_prediction[0])])

In [ ]:
import math

def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images_batch[i])
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array) 
    
    predicted_class = class_names[np.argmax(batch_prediction[0])]
    confidence = tf.reduce_max(predictions[0]).numpy()
    confidence = round(100 * (np.max(predictions[0])), 2)


    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images_batch[i].numpy().astype("uint8"))

    predicted_class, confidence = predict(model, images_batch[i].numpy())
    actual_class = class_names[labels_batch[i]]

    plt.title(f"Actual: {actual_class}, \n Predicted: {predicted_class}.\n Confidence: {confidence}%")
    
    plt.axis('off')  # Hide the axes


In [ ]:
model_version = 1
model.save(f"../models/{model_version}.keras")


In [ ]:
import os

# Find the highest model version in the directory and increment by 1
model_version = max((int(i.replace('.keras', '')) for i in os.listdir("../models") if i.endswith('.keras')), default=0) + 1

# Save the model with the new version number using the native Keras format
model.save(f"../models/{model_version}.keras")
